# Parsing System Files Example
This notebook is intended to show how the parsing of matpower, pss/e and plexos data can be done to integrate it with the PowerSystems platform. 

In [ ]:
] activate env; instantiate

In [ ]:
] st

Also, run `] build PowerSystems` (here or in a REPL) to download data necessary to run this notebook

In [ ]:
using PowerSystems
using TimeSeries
const PSY = PowerSystems
const IS = PSY.InfrastructureSystems;

- To disable PowerModels warnings 
  - Edit : PowerModels=>src=>PowerModels.jl :line 21
  - Memento.config!(LOGGER, "warn")  =>  Memento.config!(LOGGER, "error")

In [ ]:
base_dir = dirname(dirname(pathof(PowerSystems)))

## Read Data files to create a to PowerSystems `System` object

### MATPOWER

In [ ]:
sys_matpower = PSY.parse_standard_files(joinpath(base_dir, "data/matpower/RTS_GMLC.m"));

In [ ]:
sys_matpower

### PSS/E

In [ ]:
sys_psse = PSY.parse_standard_files(joinpath(base_dir,"data/psse_raw/RTS-GMLC.RAW"));

In [ ]:
sys_psse

### CDM - Tabular data format adopted by the [RTS-GMLC](https://github.com/gridmod/rts-gmlc)

*This format parses the all of time series data and assignes it to the `PowerSystemRaw` object. To generate a valid `System` we need to select one of the forecasts.*

In [ ]:
RTS_GMLC_DIR = joinpath(base_dir,"data/RTS_GMLC");

#parse in tabular data
rawsys = PSY.PowerSystemTableData(RTS_GMLC_DIR,100.0, joinpath(RTS_GMLC_DIR,"user_descriptors.yaml"))

#create an hourly model from tabular data
sys = System(rawsys; forecast_resolution = Dates.Hour(1));

In [ ]:
sys

## `System`s created from MATPOWER and PSS/E files don't automatically contain time series data

In [ ]:
FORECASTS_DIR = joinpath(base_dir,"data/forecasts/")


In [ ]:
sys_5 = PSY.parse_standard_files(joinpath(base_dir, "data/matpower", "case5_re.m"))

In [ ]:
ts_pointers = IS.read_timeseries_metadata(joinpath(FORECASTS_DIR,
                            "5bus_ts","timeseries_pointers_da.json"))

In [ ]:
add_forecasts!(sys_5, ts_pointers)
sys_5

## JSON serialization of data

In [ ]:
path, io = mktemp()
@info "Serializing to $path"
to_json(io, sys)
close(io)

In [ ]:
filesize(path)/1000000 #MB

In [ ]:
sys2 = System(path)